UNIVERSIDAD GALILEO

STATISTICAL LEARNING II

Marco Vinicio Escalante Lara

Carné: **19001148**

# PARTE No. 3

Red neuronal tipo recurrente para datos temporales y secuenciales.

Para la elaboración de este proyecto vamos a trabajar con la predicción de goles que se anotaron en un partido, por medio de seleccionar un script de un partido de futbol en ingles, se tomo el idioma ingles ya que es el que tenemos para muchas funciones y nos sirve para poder trabajar de mejor manera, así como el script se encuentra mejor colocado, para trabara la red neuronal recurrente empleamos glove, como se podra ir viendo a lo largo de este ejercicio, nos apoyamos en ciertos documentos que mostraremos a continuación.

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [104]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

Tomamos de un script del partido entre Barcelo y Real Madrid, los datos de las veces que se mencionaba que había ocurrido un gol y se tomaron veinte palabras antes y veinte palabraas despues, para determinar si hubo o no gol, este partido en cuestion quedo 5-2

In [56]:
# Ocurrencias de palabra "goal" +- 20 palabras atrás y adelante
docs = ["this locker room with this score lane the threat that his team have produced [music] still looking for the go-ahead goal but it defended as well as he could against the barcelona team that isn't doing too much wrong could",
 'come back he will tonight in his dreams now keeping a few numbers back do not want to surrender a goal before the break if they can avoid it what a sonido got the ball and neymar needed to get',
 "be the goat as plenty of superstars out there to make the difference i think there's going to be a goal or two in this second novel but real madrid who's got the bench it could perhaps make a difference",
 'ricky by [applause] do not actually scored in his first two classic rows andreessen rica followed up of a game-tying goal the next year only on the warriors today i send a change between nes that and messi been going',
 "just can't find enough rainbow on that pass to take it over the top of marcel winter capisce the breakthrough goal coming from the beheaded boy niki sanders would we're back his little boy and shakira it's a wonderful hello",
 'even for the costa rica man the greatest gear for the greatest game in the world at soccer calm second goal of the season for pk raqqa ditch puts it in perfectly here i said there was a golden ray',
 "in recent years not quite back to his sevilla days we'll pass it off this time gets messy with the goal barcelona be a bit more patient not steal against roma is not the only derby of note the classical",
 "upstairs wanted to see a little bit more magic from this classical encounter knotted up at one won't see a goal for 56 minutes we see two and seven who is never satisfied that's for sure sounding like a man",
 'air set the table for kareem and the cream rises to the top again inside for iniesta when he first goal for ben zoma tying his watermark in spain still in the race for the pichichi all those looks away',
 'juan carlos moon zuy he steps up in place of luis enrique for every set play a free-kick around the goal and also a corner that was not an accident that was purposely by design neymar came across and when',
 'accident that was purposely by design neymar came across and when all the players were celebrating after bk scored the goal javier mascherano ran across to warn carlos and zuy and congratulated him back to you phil on the defensive',
 'benzema the player coming out a different look they see different animal the content here to place been fitting wonderful goal scorer for real madrid he brought them back into this game gets the caned i trace from the world-class',
 'chance for anybody once it got over and took that deflection but he catapulted that one horn when he first goal of the season in league play for benzema now this will likely push ronaldo higher up more into that',
 'earns the whistle real question now is for real madrid how much do they want to push for a second goal as barcelona looks content to bleed another game off the schedule as he stripped conseguido again si out whines',
 "feeds i don't see anything wrong there people i'm sorry on that angle there's nothing wrong that's a legitimately grid goal by gareth from this angle and again that's a very harsh call against the mayor breathing dragon that rises",
 'zinedine zidane he has come to life in the second half here on the bench for real madrid when the goal went in he celebrated with his technical staff with the substitute with everybody and he seemed to be the',
 'with the substitute with everybody and he seemed to be the last person inside camp now that realized that the goal was disallowed it was phenomenal viewing see dan clearly furious afterwards phil thank you very much ter daughter sobrino',
 "attempt at defending from alba and gareth again so close to not making lo do bravo and getting the go-ahead goal yet again there will be plenty to argue about back in the nation's capital oh ramos going in strong",
 'always categorizes not perhaps getting along with each other only got along each other lake heckle and jeckle their wonderful goal for the lead 7th minute is neymar bursting forward looking for an equaliser top of the box lays it',
 'back messi the flex kicked off by any and cleared we still might have some change but what a huge goal for ronaldo only the eight league clásico goal he is scored but the 16th of all time only disgusting',
 'and cleared we still might have some change but what a huge goal for ronaldo only the eight league clásico goal he is scored but the 16th of all time only disgusting chance for swat has been a push and',
 "of it now it seems as though barcelona with a spark they've got a man advantage but they're down a goal and two minutes left of regulation did they take it too easy the 39 match unbeaten streak about to"]

In [57]:
# Cantidad de frases
len(docs)

22

In [72]:
# Definimos las etiquetas de gol
# Cambiar por las que representen efectivamente un gol
labels = np.random.choice(np.array([1., 0.]), size=len(docs), p=[0.25, 0.75])
labels

array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 1.])

In [76]:
# Definiciones útiles
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 25

In [73]:
# Preparar el Tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
# Obtener las secuencias de números
sequences = t.texts_to_sequences(docs)

# Vocabulario y tamaño del vocabulario
word_index = t.word_index
vocab_size = len(word_index) + 1
print("Tamaño del vocabulario: %d" % (vocab_size))

# Obtener las secuencias espaciadas (o uniformes)
data = sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre")

Tamaño del vocabulario: 394


In [74]:
data[1, :]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  77,  15,   8,
        51, 153,   9,  17, 154,  24, 155,   2, 156, 157,  15,  35,  16,
        78,   6, 158,   2,   3, 159,   1, 160, 161,  36, 162, 163,  10,
        52,   2, 164,  25,   1, 165,   5,  37, 166,   6, 167], dtype=int32)

In [100]:
word_index

{'the': 1,
 'a': 2,
 'goal': 3,
 'for': 4,
 'and': 5,
 'to': 6,
 'that': 7,
 'he': 8,
 'in': 9,
 'it': 10,
 'of': 11,
 'with': 12,
 'this': 13,
 'but': 14,
 'back': 15,
 'not': 16,
 'his': 17,
 'only': 18,
 'as': 19,
 'be': 20,
 'was': 21,
 'is': 22,
 'see': 23,
 'now': 24,
 'got': 25,
 'i': 26,
 'real': 27,
 'by': 28,
 'on': 29,
 'from': 30,
 'again': 31,
 'when': 32,
 'still': 33,
 'barcelona': 34,
 'do': 35,
 'they': 36,
 'neymar': 37,
 'there': 38,
 'two': 39,
 'second': 40,
 'madrid': 41,
 'scored': 42,
 'up': 43,
 'game': 44,
 'all': 45,
 'have': 46,
 'could': 47,
 'against': 48,
 'much': 49,
 'wrong': 50,
 'will': 51,
 'what': 52,
 'out': 53,
 'going': 54,
 'first': 55,
 'change': 56,
 'been': 57,
 'top': 58,
 'wonderful': 59,
 'man': 60,
 'at': 61,
 'here': 62,
 'off': 63,
 'time': 64,
 'more': 65,
 'we': 66,
 "that's": 67,
 'across': 68,
 'league': 69,
 'push': 70,
 'ronaldo': 71,
 'team': 72,
 'looking': 73,
 'go': 74,
 'ahead': 75,
 'too': 76,
 'come': 77,
 'want': 78,
 'ple

## Cargar el *embedding* a la memoria

In [77]:
from gensim.models import KeyedVectors

Cargamos datos de un modelo que ya contaba con registros, que permite ver nuestra información de manera vectorizada, esta información se encuentra en linea y es libre, lo que hace es pasar nuestra palabra a una forma matricial que tiene distintos pesos.

In [82]:
# Cargar el modelo Stanford Glove de Twitter
word2vec_file = '../glove-example/glove.twitter.27B.25d.word2vec'
glove_model = KeyedVectors.load_word2vec_format(word2vec_file, binary=False)

/home/rrcp/anaconda3/envs/galpython/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [90]:
glove_model['hello']

array([-0.77069  ,  0.12827  ,  0.33137  ,  0.0050893, -0.47605  ,
       -0.50116  ,  1.858    ,  1.0624   , -0.56511  ,  0.13328  ,
       -0.41918  , -0.14195  , -2.8555   , -0.57131  , -0.13418  ,
       -0.44922  ,  0.48591  , -0.6479   , -0.84238  ,  0.61669  ,
       -0.19824  , -0.57967  , -0.65885  ,  0.43928  , -0.50473  ],
      dtype=float32)

In [91]:
glove_model['<number>']

array([ 1.3956  ,  0.2892  ,  0.48572 , -1.1412  ,  0.21461 ,  1.0714  ,
        0.25408 ,  2.1181  ,  0.30252 ,  0.75955 ,  1.1299  , -0.021373,
       -3.7757  ,  0.89387 , -0.71476 , -1.6997  , -0.42166 , -0.12601 ,
       -1.2984  ,  0.41689 , -0.84993 , -1.5199  ,  0.40681 ,  0.15024 ,
        0.26997 ], dtype=float32)

In [95]:
# Crear una matriz de embedding para el vocabulario
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = glove_model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        # Las palabras no encontradas en el modelo glove serán cero
        continue

In [99]:
embedding_matrix.shape

(394, 25)

In [105]:
# Cargar la matriz de embedding en una capa de Embedding
# Configuramos trainable = False para mantener el embedding fijo
embedding_layer = Embedding(vocab_size, EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [106]:
embedding_layer

## Definir el modelo RNN con keras

In [120]:
# Definir el modelo 
model = Sequential()
model.add(embedding_layer)
#model.add(Flatten())
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Resumir el modelo
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 25)            9850      
_________________________________________________________________
lstm_6 (LSTM)                (None, 10)                1440      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 11,301
Trainable params: 1,451
Non-trainable params: 9,850
_________________________________________________________________
None


In [132]:
# Ajustar el modelo
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
history = model.fit(data, labels, epochs=100, batch_size=64, verbose=1)
history

Epoch 1/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5009 - acc: 0.7727
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4985 - acc: 0.7727
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4961 - acc: 0.7727
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4936 - acc: 0.7727
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4911 - acc: 0.7727
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4885 - acc: 0.7727
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4858 - acc: 0.7727
Epoch 8/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4831 - acc: 0.7727
Epoch 9/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4803 - acc: 0.7727
Epoch 10/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4775 - acc: 0.7727
Epoch 11/100
22/22 [==============================] - 0s 2m

Epoch 88/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1264 - acc: 1.0000
Epoch 89/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1239 - acc: 1.0000
Epoch 90/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1214 - acc: 1.0000
Epoch 91/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1189 - acc: 1.0000
Epoch 92/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1164 - acc: 1.0000
Epoch 93/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1139 - acc: 1.0000
Epoch 94/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1115 - acc: 1.0000
Epoch 95/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1093 - acc: 1.0000
Epoch 96/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1073 - acc: 1.0000
Epoch 97/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1054 - acc: 1.0000
Epoch 98/100
22/22 [==============================

In [133]:
data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,  13, 142, 143,  12,
        13, 144, 145,   1, 146,   7,  17,  72,  46, 147, 148,  33,  73,
         4,   1,  74,  75,   3,  14,  10, 149,  19, 150,  19,   8,  47,
        48,   1,  34,  72,   7, 151, 152,  76,  49,  50,  47], dtype=int32)

### Resultados

A continuación se motraran los resultados de la predicción:

In [138]:
np.column_stack((model.predict(data), labels.reshape(-1, 1)))

array([[0.06170386, 0.        ],
       [0.05452693, 0.        ],
       [0.07049164, 0.        ],
       [0.86438251, 1.        ],
       [0.83694518, 1.        ],
       [0.06357151, 0.        ],
       [0.10685599, 0.        ],
       [0.06697136, 0.        ],
       [0.06351531, 0.        ],
       [0.0554477 , 0.        ],
       [0.85882735, 1.        ],
       [0.864223  , 1.        ],
       [0.05386209, 0.        ],
       [0.0580298 , 0.        ],
       [0.83283526, 1.        ],
       [0.06520793, 0.        ],
       [0.05038138, 0.        ],
       [0.07135829, 0.        ],
       [0.05392715, 0.        ],
       [0.82507157, 1.        ],
       [0.05448366, 0.        ],
       [0.8418771 , 1.        ]])

In [144]:
# Para predecir un único valor
model.predict(data[0].reshape(1, -1))

array([[0.06170391]], dtype=float32)

## Conclusiones

Como se puede ver en los resultados, la predicción de las ocurrencias de gol fue bastante acertada.

A pesar de que estamos utilizando datos recolectados de twitter si se tiene alto indice de acierto para determinar los resultados correctos.

Glove embedding nos ayuda en gran medida para trabajar este tipo de información.
